# https://www.kaggle.com/c/digit-recognizer/
# with Tensorflow, the CPU mode took too much time on KNN.

In [142]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [143]:
config = tf.ConfigProto(device_count={"CPU": 4}, # limit to num_cpu_core CPU usage
                inter_op_parallelism_threads = 1, 
                intra_op_parallelism_threads = 4,
                log_device_placement=True)

In [144]:
# load csv files to numpy arrays

def load_data():
    train = pd.read_csv("train.csv")
    print(train.shape)
    X_train = train.values[:,1:] 
    y_train = train.values[:,0]
    
    
    Pred_test = pd.read_csv("test.csv").values
    return X_train, y_train, Pred_test

x_train, y_train, pred_test = load_data()

(42000, 785)


In [145]:
print(x_train.shape, y_train.shape, pred_test.shape)

(42000, 784) (42000,) (28000, 784)


In [146]:
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [147]:
'''pca = PCA(n_components=50)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)'''

'pca = PCA(n_components=50)\npca.fit(x_train)\nx_train = pca.transform(x_train)\nx_test = pca.transform(x_test)'

In [148]:
sess = tf.InteractiveSession(config=config)

/Users/vincent/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [149]:
'''
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
#def get_prediction(distance, k):
# Get k-min distance index (Nearest neighbor)
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
prediction_indices = tf.gather(y_train, top_k_indices)

y, idx, cnts = tf.unique_with_counts(prediction_indices,)
max_index = tf.argmax(cnts) #vote it from k

# Predict the mode category
#    return y[max_index]
'''

'\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\n#def get_prediction(distance, k):\n# Get k-min distance index (Nearest neighbor)\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\nprediction_indices = tf.gather(y_train, top_k_indices)\n\ny, idx, cnts = tf.unique_with_counts(prediction_indices,)\nmax_index = tf.argmax(cnts) #vote it from k\n\n# Predict the mode category\n#    return y[max_index]\n'

In [150]:
k = 5
batch_size=100

# Placeholders
x_data_train = tf.placeholder(shape=x_train.shape, dtype=tf.float32)
x_data_test = tf.placeholder(shape=[None, 784], dtype=tf.float32)
y_target_train = tf.placeholder(shape=y_train.shape, dtype=tf.float32)
y_target_test = tf.placeholder(shape=[batch_size], dtype=tf.float32)

distance = tf.reduce_sum(
    tf.abs(
        tf.subtract(x_data_train, tf.expand_dims(x_data_test, 1))
    ), axis=2)

# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 1)

prediction = tf.gather(y_target_train, pred)
#correct_prediction = tf.equal(prediction, y_target_test)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Calculate how many loops over training data
num_loops = int(np.ceil(len(pred_test)/batch_size))

test_output = []
actual_vals = []
for i in range(num_loops):
    min_index = i*batch_size
    max_index = min((i+1)*batch_size,len(x_train))
    print(max_index)
    
    x_batch = pred_test[min_index:max_index]
    #x_batch = x_test[min_index:max_index]
    #y_batch = y_test[min_index:max_index]
    predictions = sess.run(prediction, feed_dict={x_data_train: x_train, 
                                                  x_data_test: x_batch,
                                                  y_target_train: y_train
                                                  #y_target_test: y_batch
                                                 })
    test_output.extend(predictions)
    #actual_vals.extend(y_batch)




#correct_prediction = tf.equal(test_output, actual_vals)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy on test set: ' + str(accuracy))

def flush_predictons(predictions):
    imgs = pd.Series(range(1,pred_test.shape[0]+1))
    imgd = pd.DataFrame(imgs)
    
    labeld = pd.DataFrame(predictions)
    out = pd.concat([imgd, labeld],axis=1)
    out.columns = ["ImageId", "Label"]
    out.to_csv("KNN_PCA_Prediction.csv", index = False)

flush_predictons(test_output);

100
200
300
400
500
600
700
800


KeyboardInterrupt: 